# Open multiples datasets

Some safe files have multiples subdatasets, like WV or IW_SLC. This notebook show how to handle them, or how to handle multiple datasets.

In [ ]:
import xsar 
import xarray as xr
import glob
import holoviews as hv
import geoviews as gv
hv.extension('bokeh')


SAFES containings multiples subdatasets like WV_SLC cannot be openned with `xsar.open_dataset` given the safe path.

In [ ]:
try:
    wv_slc_ds = xsar.open_dataset(xsar.get_test_file('S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_Z010.SAFE'))
except IndexError as e:
    print(str(e))

An `IndexError` is raised, with an advice to use [xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta) to get the available subdatasets list.

In [ ]:
wv_slc_meta = xsar.Sentinel1Meta(xsar.get_test_file('S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_Z010.SAFE'))
wv_slc_meta

Note that the abose `wv_slc_meta` object is a **multi dataset**

Subdatasets list is available with [xsar.Sentinel1Meta.subdatasets](../basic_api.rst#xsar.Sentinel1Meta.subdatasets) property

In [ ]:
wv_slc_meta.subdatasets

A subdataset can be directly be opened by [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

In [ ]:

wv_slc_ds1 = xsar.open_dataset(wv_slc_meta.subdatasets.index[0])
wv_slc_ds1

Or we can get a **single dataset** by using [xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta)

In [ ]:
wv_slc_meta1 = xsar.Sentinel1Meta(wv_slc_meta.subdatasets.index[0])
wv_slc_meta1

And use this `wv_slc_meta1` object with [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

In [ ]:
wv_slc_ds1 = xsar.open_dataset(wv_slc_meta1)

## Getting all xsar.Sentinel1Meta  objects from a list of path

[xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta) can only handle one path at a time.

With [xsar.product_info](../basic_api.rst#xsar.product_info), you can provide a list of path, and get a `pandas.DataFrame` with all `meta` objects.

In [ ]:
df_info = xsar.product_info([
    xsar.get_test_file('S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_Z010.SAFE'),
    xsar.get_test_file('S1A_IW_SLC__1SDV_20170907T102951_20170907T103021_018268_01EB76_Z010.SAFE')
])
df_info

By default,  [xsar.product_info](../basic_api.rst#xsar.product_info) only return a minimal set of columns.

Columns list can be given by the `columns=` keyword. 

Columns name can be any attributes from [xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta), with the addition of the 'meta' column name, who refer to the [xsar.Sentinel1Meta](../basic_api.rst#xsar.Sentinel1Meta) objet itself.

If the columns list include 'geometry', the result will by a `geopandas.GeoDataframe`, who can be displayed whith holoviews.

In [ ]:
gdf_info = xsar.product_info(
    [
        xsar.get_test_file('S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_Z010.SAFE'),
        xsar.get_test_file('S1A_IW_SLC__1SDV_20170907T102951_20170907T103021_018268_01EB76_Z010.SAFE')
        
    ],
    columns=[ 'safe', 'dsid', 'meta', 'geometry' ]
)

# 'meta' column is dropped for display
gv.feature.land * gv.Polygons(gdf_info.drop(['meta'], axis=1)).opts(tools=['hover'])


In [ ]:
gdf_info.iloc[0]['meta']